In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np


In [2]:
## Primeiro tabulamos os tipos de ocupações existentes na base do CNES e agrpamos


query = "SELECT ano, tipo_ocupacao,sigla_uf, sum(n_profissionais) as quantidade_vinculos  FROM `basedosdados-projetos.republica.vinculo_saude_tipo_ocupacao` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf"

In [3]:
df=bd.read_sql(query, billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 49342/49342 [00:03<00:00, 13159.03rows/s]


In [4]:
df[df['tipo_ocupacao'].str.startswith('Médicos')]['tipo_ocupacao'].unique()

array(['Médicos', 'Médicos em especialidades cirúrgicas',
       'Médicos clínicos',
       'Médicos em medicina diagnóstica e terapêutica'], dtype=object)

In [23]:
df[df['tipo_ocupacao'].str.startswith('Médicos')]['tipo_ocupacao'].unique()
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.startswith('Médicos'),'Médicos',df['tipo_ocupacao'])
## Aqui eu unifico os três tipos de Médicos (clínica, cirurgia e terapeutico)

In [24]:
df.sort_values('quantidade_vinculos',ascending=False)

,ano,tipo_ocupacao,sigla_uf,quantidade_vinculos,tipo_2
45092,2021,Técnicos e auxiliares de enfermagem,SP,115763,Técnicos e auxiliares de enfermagem
40514,2020,Técnicos e auxiliares de enfermagem,SP,104659,Técnicos e auxiliares de enfermagem
27720,2017,Técnicos e auxiliares de enfermagem,SP,103317,Técnicos e auxiliares de enfermagem
31847,2018,Técnicos e auxiliares de enfermagem,SP,102876,Técnicos e auxiliares de enfermagem
36082,2019,Técnicos e auxiliares de enfermagem,SP,101036,Técnicos e auxiliares de enfermagem
...,...,...,...,...,...
40163,2019,Engenheiros agrossilvipecuários,RN,1,Engenheiros agrossilvipecuários
21852,2015,Profissionais polivalentes da confecção de roupas,ES,1,Profissionais polivalentes da confecção de roupas
40165,2019,"Lavadores e passadores de roupa, a mão",RO,1,"Lavadores e passadores de roupa, a mão"
21839,2015,Professores de ciências biológicas e da saúde ...,ES,1,Professores de ciências biológicas e da saúde ...


In [83]:
list(df.head(17)['tipo_ocupacao'])

['Assistentes sociais e economistas domésticos',
 'Médicos clínicos',
 'Cirurgiões-dentistas',
 'Enfermeiros e afins',
 'Enfermeiros e afins',
 'Cirurgiões-dentistas',
 'Médicos clínicos',
 'Assistentes sociais e economistas domésticos',
 'Médicos clínicos',
 'Psicólogos e psicanalistas',
 'Técnicos e auxiliares de enfermagem',
 'Técnicos e auxiliares de enfermagem',
 'Técnico em farmácia e em manipulação farmacêutica',
 'Tecnólogos e técnicos em métodos de diagnósticos e terapêutica',
 'Técnicos de odontologia',
 'Enfermeiros e afins',
 'Engenheiros eletricistas, eletrônicos e afins']

In [25]:
ocupacao_interesse = ['Técnicos e auxiliares de enfermagem',
 'Médicos',
 'Trabalhadores em serviços de promoção e apoio à saúde',
 'Enfermeiros e afins',
 'Cirurgiões-dentistas',
 'Fisioterapeutas',
 'Psicólogos e psicanalistas',
 'Farmacêuticos',
 'Técnicos de odontologia',
 'Tecnólogos e técnicos em métodos de diagnósticos e terapêutica',
 'Nutricionistas',
 'Assistentes sociais e economistas domésticos','Outros']

In [26]:
df['tipo_3'] = np.where(df['tipo_2'].isin(ocupacao_interesse) ,df['tipo_2'],'Outros')

In [99]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,quantidade_vinculos,tipo_2,tipo_3
0,2008,Psicólogos e psicanalistas,DF,107,Psicólogos e psicanalistas,Psicólogos e psicanalistas
1,2008,Nutricionistas,DF,238,Nutricionistas,Nutricionistas
2,2008,Enfermeiros e afins,DF,1527,Enfermeiros e afins,Enfermeiros e afins
3,2008,Cirurgiões-dentistas,RR,144,Cirurgiões-dentistas,Cirurgiões-dentistas
4,2008,Fonoaudiólogos,DF,33,Fonoaudiólogos,Outros
5,2008,Veterinários e zootecnistas,DF,3,Veterinários e zootecnistas,Outros
6,2008,Técnicos de odontologia,DF,113,Técnicos de odontologia,Técnicos de odontologia
7,2008,Físicos,DF,3,Físicos,Outros
8,2008,Técnicos e auxiliares de enfermagem,DF,7596,Técnicos e auxiliares de enfermagem,Técnicos e auxiliares de enfermagem
9,2008,Trabalhadores em serviços de promoção e apoio ...,DF,1204,Trabalhadores em serviços de promoção e apoio ...,Trabalhadores em serviços de promoção e apoio ...


In [27]:
x=df.pivot_table(index=['ano','sigla_uf'  ,'tipo_3'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
y = y.reset_index() ##inserir a contagem como coluna
#y = y.rename(columns={0:'count'})

In [30]:
y

,ano,sigla_uf,tipo_3,quantidade_vinculos
0,2008,AC,Assistentes sociais e economistas domésticos,25
1,2008,AC,Cirurgiões-dentistas,110
2,2008,AC,Enfermeiros e afins,343
3,2008,AC,Farmacêuticos,18
4,2008,AC,Fisioterapeutas,46
...,...,...,...,...
4908,2021,TO,Psicólogos e psicanalistas,471
4909,2021,TO,Tecnólogos e técnicos em métodos de diagnóstic...,395
4910,2021,TO,Trabalhadores em serviços de promoção e apoio ...,5237
4911,2021,TO,Técnicos de odontologia,651


In [131]:
7966/2881900*1000

2.7641486519310177

In [31]:
## Ajeitando para o BQ

df= y.rename(columns={'tipo_3':'tipo_ocupacao'})

In [34]:
df.sort_values('quantidade_vinculos', ascending=False)

,ano,sigla_uf,tipo_ocupacao,quantidade_vinculos
4892,2021,SP,Médicos,132869
4894,2021,SP,Outros,125083
4541,2020,SP,Médicos,116414
4899,2021,SP,Técnicos e auxiliares de enfermagem,115763
3839,2018,SP,Médicos,112194
...,...,...,...,...
2729,2015,RR,Assistentes sociais e economistas domésticos,6
2379,2014,RR,Assistentes sociais e economistas domésticos,5
2385,2014,RR,Nutricionistas,5
2387,2014,RR,Psicólogos e psicanalistas,4


In [35]:
df = df[df['ano']>2012]

In [36]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4913 entries, 0 to 4912
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano                  4913 non-null   Int64 
 1   sigla_uf             4913 non-null   object
 2   tipo_ocupacao        4913 non-null   object
 3   quantidade_vinculos  4913 non-null   Int64 
dtypes: Int64(2), object(2)
memory usage: 163.3+ KB


In [37]:


schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos')
 ]


In [38]:
table_ref = dataset_ref.table('CNES_total_profissionais_saude')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=37c3dbbf-4f28-4e7e-9b71-03cb82bad6ab>